In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
import jax
import jax.numpy as jnp
from llama import flax_model, model
import torch
import numpy as np
from fairscale.nn.model_parallel.initialize import initialize_model_parallel
from typing import Tuple, List, Tuple
import os
from flax.core.frozen_dict import unfreeze, freeze
from dataclasses import dataclass, asdict
import functools
from example import load
from convert_weights import convert_llama_weights
from llama.tokenizer import Tokenizer

In [3]:
np.random.seed(0)
ckpt_dir = '/shared/csnell/llama_weights/7B/'
tokenizer_path = '/shared/csnell/llama_weights/tokenizer.model'
tokenizer = Tokenizer(model_path=tokenizer_path)
# params = {"dim": 4096, "multiple_of": 256, "n_heads": 32, "n_layers": 32, "norm_eps": 1e-06, "vocab_size": tokenizer.n_words, "max_batch_size": 32, "max_seq_len": 1024}

In [4]:
# jax_model = flax_model.Transformer(**params)
# jax_state = jax_model.init(
#     jax.random.PRNGKey(0), 
#     jnp.ones((1, 1), dtype=jnp.int32), 
#     0, 
# )
jax_weights, params = convert_llama_weights(ckpt_dir)

In [7]:
params.update({'max_batch_size': 32, 'max_seq_len': 1024, 'vocab_size': tokenizer.n_words})

In [10]:
jax_state = {
    'params': jax_weights, 
    'cache': {
        'layer_%d' % (i): {
            'attention': {
                'cache_k': jnp.zeros((params['max_batch_size'], params['max_seq_len'], params['n_heads'], params['dim'] // params['n_heads'])), 
            }, 
        } for i in range(params['n_layers'])
    }, 
}

In [17]:
test_strs = [
    "The capital of Germany is the city of", 
    "Here is my sonnet in the style of Shakespeare about an artificial intelligence:", 
]
tokens = [tokenizer.encode(x, bos=True, eos=False) for x in test_strs]

[[1, 450, 7483, 310, 9556, 338, 278, 4272, 310],
 [1,
  2266,
  338,
  590,
  1487,
  1212,
  297,
  278,
  3114,
  310,
  23688,
  1048,
  385,
  23116,
  21082,
  29901]]

In [15]:
jax_model = flax_model.Transformer(**params)

In [16]:
jax_model.apply(
    jax_state, 
    jnp.asarray(tokens[0][:params['max_seq_len']])[None], 
    0, 
    mutable=['cache'], 
)

2023-03-06 19:05:54.701329: W external/org_tensorflow/tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 512.00MiB (rounded to 536870912)requested by op 
2023-03-06 19:05:54.701721: W external/org_tensorflow/tensorflow/tsl/framework/bfc_allocator.cc:497] *********************************************_******************************************************
2023-03-06 19:05:54.702089: E external/org_tensorflow/tensorflow/compiler/xla/pjrt/pjrt_stream_executor_client.cc:2410] Execution of replica 0 failed: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 536870912 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:  512.14MiB
              constant allocation:        16B
        maybe_live_out allocation:  512.00MiB
     preallocated temp allocation:   144.0KiB
  preallocated temp fragmentation:         0B (0.00%)
                 total allocation:    1.00GiB
Peak buffers:
	Buffer 

XlaRuntimeError: RESOURCE_EXHAUSTED: Out of memory while trying to allocate 536870912 bytes.
BufferAssignment OOM Debugging.
BufferAssignment stats:
             parameter allocation:  512.14MiB
              constant allocation:        16B
        maybe_live_out allocation:  512.00MiB
     preallocated temp allocation:   144.0KiB
  preallocated temp fragmentation:         0B (0.00%)
                 total allocation:    1.00GiB
Peak buffers:
	Buffer 1:
		Size: 512.00MiB
		Entry Parameter Subshape: f32[32,1024,32,128]
		==========================

	Buffer 2:
		Size: 512.00MiB
		XLA Label: copy
		Shape: f32[32,1024,32,128]
		==========================

	Buffer 3:
		Size: 144.0KiB
		XLA Label: fusion
		Shape: f32[1,9,32,128]
		==========================

	Buffer 4:
		Size: 144.0KiB
		Entry Parameter Subshape: f32[1,9,32,128]
		==========================

	Buffer 5:
		Size: 16B
		XLA Label: constant
		Shape: s32[4]
		==========================

	Buffer 6:
		Size: 8B
		Entry Parameter Subs